# Aprenentatge Supervisat - Regressions

**Descripció**

Anem a practicar i a familiaritzar-nos amb regressions

In [1]:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import time

from sklearn.preprocessing import StandardScaler, RobustScaler 
from sklearn.preprocessing import PowerTransformer, OneHotEncoder
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm

from utils import SinCosTransformer

import warnings
warnings.simplefilter('ignore')

SEED = 42
TEST_SIZE = 0.2
TESTING = True

### Carreguem les dades

In [2]:
dfdelays = pd.read_pickle('../data/S11-CleanDelayedFlights.pickle')

In [3]:
# Fem un sample de les dades per fer el procés més ràpid 

if TESTING:
    dfdelays = dfdelays.sample(frac=0.05, random_state=SEED)
    
dfdelays.shape

(96419, 87)

### Separem en train test

In [4]:
target = ['ArrDelay']

cols = [col for col in dfdelays.columns if col not in target]

X_train, X_test, y_train, y_test = train_test_split(dfdelays[cols], dfdelays[target], test_size=TEST_SIZE, random_state=SEED )

In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((77135, 86), (19284, 86), (77135, 1), (19284, 1))

### Preprocés

In [6]:
# columnes originals

base_cols = ['Year', 'Month', 'DayofMonth', 'DayOfWeek', 
             'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime', 
             'UniqueCarrier', 'AirTime', 'DepDelay', 
             'Origin', 'Dest', 'Distance', 'TaxiIn', 'TaxiOut', 
             'CRSElapsedTime_c', 'ActualElapsedTime_c']


In [7]:
cols_exp4 = ['ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 
            'DepDelay', 'Origin', 'Dest', 'UniqueCarrier', 'Distance', 
            'carrier_name', 'Name_Origin', 'Name_Dest', 
            'CRSElapsedTime_c', 'ActualElapsedTime_c', 
            'DepartureTime', 'CRSDepartureTime', 
            'ArrivalTime', 'CRSArrivalTime', 
            'Dep_hourlyf', 'Dep_dailyf', 'Arr_dailyf', 'Arr_hourlyf',    
            'Origin_lat', 'Origin_long', 'Dest_lat', 'Dest_long', 
            'velocity', 'CRSvelocity', 
            'DepartureTime_hour_sin', 'DepartureTime_hour_cos', 'DepartureTime_day_sin', 'DepartureTime_day_cos', 
            'DepartureTime_dayofweek_sin', 'DepartureTime_dayofweek_cos', 'DepartureTime_month_sin', 'DepartureTime_month_cos', 
            'CRSDepartureTime_hour_sin', 'CRSDepartureTime_hour_cos', 'CRSDepartureTime_day_sin', 'CRSDepartureTime_day_cos', 
            'CRSDepartureTime_dayofweek_sin', 'CRSDepartureTime_dayofweek_cos', 'CRSDepartureTime_month_sin', 'CRSDepartureTime_month_cos', 
            'ArrivalTime_hour_sin', 'ArrivalTime_hour_cos', 'ArrivalTime_day_sin', 'ArrivalTime_day_cos', 
            'ArrivalTime_dayofweek_sin', 'ArrivalTime_dayofweek_cos', 'ArrivalTime_month_sin', 'ArrivalTime_month_cos', 
            'CRSArrivalTime_hour_sin', 'CRSArrivalTime_hour_cos', 'CRSArrivalTime_day_sin', 'CRSArrivalTime_day_cos', 
            'CRSArrivalTime_dayofweek_sin', 'CRSArrivalTime_dayofweek_cos', 'CRSArrivalTime_month_sin', 'CRSArrivalTime_month_cos']


In [8]:
# amb columes noves
cols_exp5 = ['DepDelay',  'Distance', 
            'UniqueCarrier', 
            'CRSElapsedTime_c', 'ActualElapsedTime_c', 
            'DepartureTime', 'CRSDepartureTime', 
            'ArrivalTime', 'CRSArrivalTime', 
            'Dep_hourlyf', 'Dep_dailyf', 'Arr_dailyf', 'Arr_hourlyf',    
            'Origin_lat', 'Origin_long', 'Dest_lat', 'Dest_long', 
            'velocity', 'CRSvelocity']

# Eliminem el DepDelay
cols_exp6 = ['Distance', 
            'UniqueCarrier', 
            'CRSElapsedTime_c', 'ActualElapsedTime_c', 
            'DepartureTime', 'CRSDepartureTime', 
            'ArrivalTime', 'CRSArrivalTime', 
            'Dep_hourlyf', 'Dep_dailyf', 'Arr_dailyf', 'Arr_hourlyf',    
            'Origin_lat', 'Origin_long', 'Dest_lat', 'Dest_long', 
            'velocity', 'CRSvelocity']

# Eliminem el DepDelay, així com dades sobre el vol real, fins i tot el DepartureTime
# No esperem un bon resultat 
cols_exp7 = ['Distance', 
            'UniqueCarrier', 'Origin', 'Dest',
            'CRSElapsedTime_c', 
            'CRSDepartureTime', 
            'CRSArrivalTime', 
            'Dep_hourlyf', 'Dep_dailyf', 'Arr_dailyf', 'Arr_hourlyf',    
            'Origin_lat', 'Origin_long', 'Dest_lat', 'Dest_long', 
            'CRSvelocity']

## Nivell 1

### Exercici 1
Crea almenys tres models de regressió diferents per intentar predir el millor possible l’endarreriment dels vols (ArrDelay) de DelayedFlights.csv.

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, Ridge, HuberRegressor, GammaRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

from xgboost import XGBRegressor

# metriques
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Crearem aquests models:
- LinearRegression
- Rigde
- HuberRegressor (ourlier-robust)
- DecisionTreeRegressor
- RandomForestRegressor
- GradientBoostingRegressor
- MLPRegressor
- XGBoostRegressor


##### Preparem un parell de funcions on creem un pipeline i executem la regressió

In [10]:
def make_pipeline(data, columns, model):
    cols_num = data[columns].select_dtypes(include=['int','float']).columns.to_list()
    cols_cat = data[columns].select_dtypes(include=['object']).columns.to_list()
    cols_date = data[columns].select_dtypes(include=['datetime']).columns.to_list()
    
    ct = ColumnTransformer(
             transformers=[('num', RobustScaler(), cols_num),
                          ('cat', OneHotEncoder(handle_unknown='ignore'), cols_cat),
                          ('date', SinCosTransformer(['hour','day','dayofweek','month']), cols_date )])
    
    pipeline = Pipeline( steps=[('transformer', ct),
                               ('regr', model)])
    
    return pipeline

# base_pipe = make_pipeline(dfdelays, base_cols, LinearRegression())

# base_pipe.fit(dfdelays[base_cols], dfdelays[target])
# y_pred_base = base_pipe.predict(dfdelays[base_cols])
# print(f'LR r2: {r2_score(dfdelays[target], y_pred_base)}')
# print(f'LR MSE: {mean_squared_error(dfdelays[target], y_pred_base)}')

def run_experiment(train, test=None, columns=[], model=LinearRegression()):
    if test is None:
        X_train, X_test, y_train, y_test = train_test_split(train, 
                                                        test_size=TEST_SIZE, random_state=SEED)
    else:
        X_train, y_train = train
        X_test, y_test = test
        
    pipe = make_pipeline(X_train, columns, model)
    
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    return mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)
    
    

In [11]:
models = {'lr': LinearRegression(),
         'ridge': Ridge(),
         'hub': HuberRegressor(),
         'tree': DecisionTreeRegressor(max_depth=7, random_state=SEED),
         'rf': RandomForestRegressor(max_depth=7, n_jobs=-1, random_state=SEED),
         'gb': GradientBoostingRegressor( random_state=SEED, criterion='mse'),
         'nn': MLPRegressor(hidden_layer_sizes=(100,50,25)),
         'xgb': XGBRegressor(n_jobs=-1)}


# params = {  'lr': {'fit_intercept':False, 'normalize': True},
#             'ridge': {'alpha':np.linspace(), 'solver':{'auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga'}},
#             'hub': {},
#             'tree': {},
#             'rf': {} 
#             'gb': {} 
#             'nn': {} 
#             'xgb': {}
#          }


#### Experiments - Features escollits

- **Exp. min**: Només amb DepDelay
- **base**: Amb les dades originals netejades
- **exp. 3**: Amb només 'DepDelay', 'Distance','Origin', 'Dest', 'UniqueCarrier'
- **exercici 5**. Noves columnes. Velocitat, Latitut i Longitut dels aeroports, Numero de vols per hora i dia a l'aeroport de Sortida i Arribada
- **ND exp. 6**: com el 5 però sense el DepDelay
- **ND exp. 7**: Eliminem el DepDelay, així com dades sobre el vol real, fins i tot el DepartureTime. No esperem un bon resultat Experiments sense DepDelay

In [12]:
experiments = { 'Exp. min': ['DepDelay'],
              'base': base_cols, 
              'exp. 3': ['DepDelay', 'Distance','Origin', 'Dest', 'UniqueCarrier'],
              'exercici 5': cols_exp5,
               'ND exp. 6': cols_exp6,
               'ND exp. 7': cols_exp7 }

In [13]:
results = []

for exp, cols in tqdm(experiments.items()):
    for name, model in tqdm(models.items()):
#         print(f'{exp}: {model.__class__.__name__}')
        t0 = time.time()
        mse, r2 = run_experiment((X_train, y_train), (X_test, y_test), cols, model)
        results.append([name, mse, r2, len(X_train.shape[-1]) ,time.time()-t0, exp])
#         print(f'{exp} {name}: {model.__class__.__name__} \n\tMSE: {mse}\n\tR2 : {r2}')

dfresults =  pd.DataFrame(results, columns=['model','mse', 'r2', 'num_feat','time', 'experiment'])
# dfresults

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [14]:
dfresults.pivot(index=['model'],values=['mse','r2','num_feat','time'],columns=['experiment'])\
            .style.format("{:.5f}")\
            .set_properties(**{'background-color':'lightyellow'}, subset=['mse'])\
            .set_properties(**{'background-color':'lightgreen'}, subset=['r2'])\
            .set_properties(**{'background-color':'cyan'}, subset=['time'])\
            .highlight_min(axis=0, subset=['mse','time'])\
            .highlight_max(axis=0, subset=['r2'])


### Exercici 2
Compara’ls en base al MSE i al R2.

<div class="alert-info">
Fet durant l'exercici anterior
</div>


<!-- <span style="color:red">Mirar l'exercici anterior</span> -->

### Exercici 3
Entrena’ls utilitzant els diferents paràmetres que admeten.

In [15]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import KFold

model = XGBRegressor()

grid = {
        'regr__objective': ['reg:squarederror'],
        'regr__colsample_bytree': [0.2, 0.5, 1],
#         'regr__subsample': [0.7, 1],
#         'regr__learning_rate': [0.05, 0.1, 0.3],
        'regr__max_depth': [3, 6, 8],
        'regr__min_child_weight': [0, 1, 10],
        'regr__n_estimators' : [700, 1000, 5000]
    }

fold = KFold(n_splits=3)

pipe = make_pipeline(X_train, cols_exp4, model)

In [16]:
grid = RandomizedSearchCV(pipe, grid, n_iter=10,
                scoring=['neg_mean_squared_error','r2'], cv=fold,
                verbose=2, refit='r2', n_jobs=-1)



In [17]:
%%time 
grid.fit(X_train,y_train)

Fitting 3 folds for each of 10 candidates, totalling 30 fits


KeyboardInterrupt: 

In [18]:
print(f'GRID: \nr2:{r2_score(y_test, grid.predict(X_test))} \
        mse: {mean_squared_error(y_test, grid.predict(X_test))}')

NotFittedError: This RandomizedSearchCV instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
grid.best_params_

### Exercici 4
Compara el seu rendiment utilitzant l’aproximació traint/test o utilitzant totes les dades (validació interna)

#### CV

In [ ]:
from sklearn.metrics import SCORERS
SCORERS.keys()

In [ ]:
results = []

for exp, cols in tqdm(experiments.items()):
    for name, model in tqdm(models.items()):
#         print(f'{exp}: {model.__class__.__name__}')
        t0 = time.time()
        mse, r2 = run_experiment((X_train, y_train), (X_test, y_test), cols, model)
        results.append([name, mse, r2, len(cols) ,time.time()-t0, exp])
#         print(f'{exp} {name}: {model.__class__.__name__} \n\tMSE: {mse}\n\tR2 : {r2}')

dfresults =  pd.DataFrame(results, columns=['model','mse', 'r2', 'num_feat','time', 'experiment'])
# dfresults

In [ ]:
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score, cross_validate

cv = 5
X = dfdelays
y = dfdelays[target]

results = []
# models = {'xgb': XGBRegressor(n_jobs=-1)}
for exp, cols in tqdm(experiments.items()):
    for name, model in tqdm(models.items()):
        t0 = time.time()
#         print(model)
        pipe = make_pipeline(X, cols, model )
        score = cross_validate(pipe, X[cols], y, cv=cv, 
                               scoring=['neg_mean_squared_error','r2'],
                               n_jobs=2, verbose=0,  )
    #     results[model] = [scores, scores.mean, t]
        results.append([name, 
                        score['test_neg_mean_squared_error'].mean(),
                        score['test_r2'].mean(),
                        len(cols), 
                        score['fit_time'].sum(),
                        exp])
    

In [ ]:
dfcvresults = pd.DataFrame(results, 
                           columns=['model','cv_neg_mse','cv_r2', 'cv_time','experiment'])\


In [ ]:
dfcvresults

In [ ]:
dfresults.pivot(index=['model'],values=['mse','r2','num_feat','time'],columns=['experiment'])\
            .style.format("{:.5f}")\
            .set_properties(**{'background-color':'lightyellow'}, subset=['mse'])\
            .set_properties(**{'background-color':'lightgreen'}, subset=['r2'])\
            .set_properties(**{'background-color':'cyan'}, subset=['time'])\
            .highlight_min(axis=0, subset=['mse','time'])\
            .highlight_max(axis=0, subset=['r2'])


## Nivell 2
### Exercici 5
Realitza algun procés d’enginyeria de variables per millorar-ne la predicció

<div class="alert-info">
    <strong>Info: </strong>Resultat Experiment 5 </div>


## Nivell 3
### Exercici 6
No utilitzis la variable DepDelay a l’hora de fer prediccions

<div class="alert-info">
    <strong>Info: </strong>Resultat Experiments 6 i 7</div>



In [ ]:
sns.histplot(dfdelays.DepDelay)

In [ ]:
dfdelays.DepDelay.describe()